# Importing required functions

In [ ]:
# Import the necessary library for making API calls
%pip install requests
%pip install pandas
%pip install datetime
%pip install openpyxl
%pip install textblob
%pip install transformers
%pip install torch
%pip install plotly
%pip install matplotlib
%pip install sklearn
import requests
import pandas as pd
import datetime as dt
import numpy as np
from textblob import TextBlob
import plotly.graph_objs as go
import plotly.io as pio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ------------ --------------------------- 3.1/9.9 MB 17.3 MB/s eta 0:00:01
   ------------------------- -------------- 6.3/9.9 MB 16.2 MB/s eta 0:00:01
   ------------------------------------ --- 8.9/9.9 MB 15.1 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 13.4 MB/s  0:00:00

   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   -------------------- ------------------- 1/2 [plotly]
   -------------------- -------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Define API Key

In [ ]:
API_Eodhd = "66c0aeb1357b15.87356825"

# Creating all EODHD Functions

In [ ]:
class EODHD_Functions:
    @staticmethod
    def Fetch_exchanges_list(api_token=API_Eodhd, filtered_columns=["Name", "Code", "Country", "Currency"]):
        url = f'https://eodhd.com/api/exchanges-list/?api_token={api_token}&fmt=json'
        response = requests.get(url).json()
        exchanges_df = pd.DataFrame(response)
        if filtered_columns:
            exchanges_df = exchanges_df[filtered_columns]
        return exchanges_df

    @staticmethod
    def Fetch_exchange_symbols(exchange_code, api_token=API_Eodhd):
        url = f'https://eodhd.com/api/exchange-symbol-list/{exchange_code}?api_token={api_token}&fmt=json'
        response = requests.get(url).json()
        symbols_df = pd.DataFrame(response)
        return symbols_df
        
    
    @staticmethod
    def Fetch_index_symbols(api_token=API_Eodhd):
        url = f'https://eodhd.com/api/exchange-symbol-list/INDX?api_token={api_token}&fmt=json'
        response = requests.get(url).json()
        symbols_df = pd.DataFrame(response)
        return symbols_df
        

    @staticmethod
    def Fetch_historical_price(stock, begin_date, end_date, period, api_token=API_Eodhd, columns=["adjusted_close"]):
        url = f'https://eodhd.com/api/eod/{stock}?from={begin_date}&to={end_date}&period={period}&api_token={api_token}&fmt=json'
        response = requests.get(url).json()
        pandas_df = pd.DataFrame(response)
        if columns:
            filtered_columns = ['date'] + columns
            pandas_df = pandas_df[filtered_columns]
        return pandas_df

    @staticmethod
    def Calculate_daily_returns(stock, begin_date, end_date, period, api_token=API_Eodhd):
        historical_data = EODHD_Functions.Fetch_historical_price(stock, begin_date, end_date, period, api_token, columns=["adjusted_close"])
        historical_data['daily_return'] = np.log(historical_data['adjusted_close'] / historical_data['adjusted_close'].shift(1))
        historical_data = historical_data.dropna()
        return historical_data[['date', 'daily_return']]
    
    @staticmethod
    def Calculate_daily_log_returns(stock, begin_date, end_date, period, api_token=API_Eodhd):
        historical_data = EODHD_Functions.Fetch_historical_price(
            stock, begin_date, end_date, period, api_token, columns=["adjusted_close"]
        )
        # Calculate daily log returns
        historical_data['daily_log_return'] = np.log(
            historical_data['adjusted_close'] / historical_data['adjusted_close'].shift(1)
        )
        historical_data = historical_data.dropna()
        return historical_data[['date', 'daily_log_return']]
    
    @staticmethod
    def Fetch_news_data(stock, begin_date, end_date, offset=0, api_token=API_Eodhd):
        url = f'https://eodhd.com/api/news?s={stock}&offset={offset}&limit=1000&api_token={api_token}&from={begin_date}&to={end_date}&fmt=json'
        response = requests.get(url).json()
        news_df = pd.DataFrame(response)
        return news_df

    @staticmethod
    def Fetch_fundamentals(stock, api_token=API_Eodhd):
        url = f'https://eodhd.com/api/fundamentals/{stock}?api_token={api_token}&fmt=json'
        response = requests.get(url).json()
        fundamentals_df = pd.json_normalize(response)
        return fundamentals_df

    @staticmethod
    def Fetch_dividends(stock, start_date, end_date, api_token=API_Eodhd):
        url = f'https://eodhd.com/api/div/{stock}?from={start_date}&to={end_date}&api_token={api_token}&fmt=json'
        response = requests.get(url).json()
        fundamentals_df = pd.json_normalize(response)
        return fundamentals_df     

    @staticmethod
    def Fetch_macro_indicator(country, indicator, api_token=API_Eodhd):
        url = f'https://eodhd.com/api/macro-indicator/{country}?indicator={indicator}&api_token={api_token}&fmt=json'
        response = requests.get(url).json()
        macro_df = pd.DataFrame(response)
        return macro_df

    @staticmethod
    def Fetch_macroecon_data(code,api_token=API_Eodhd):
        url = f'https://eodhd.com/api/eod/{code}?api_token={api_token}&fmt=json'
        response = requests.get(url).json()
        macro_df = pd.DataFrame(response)
        return macro_df

    @staticmethod
    def Fetch_intraday_data(stock, interval = '5m', api_token=API_Eodhd):
        url = f'https://eodhd.com/api/intraday/{stock}?interval={interval}&split-dt=1&api_token={api_token}&fmt=json'
        response = requests.get(url)
        data = response.json()
        df = pd.DataFrame(data)
        if 'datetime' in df.columns:
            df['datetime'] = pd.to_datetime(df['datetime'])
        return df

    @staticmethod
    def Fetch_live_data(stock, api_token=API_Eodhd):
        url = f'https://eodhd.com/api/real-time/{stock}?api_token={api_token}&fmt=json'
        response = requests.get(url)
        data = response.json()
        # Convert dict to DataFrame
        df = pd.DataFrame([data])
        # Convert timestamp to datetime, and split into date and time columns
        if 'timestamp' in df.columns:
            df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
            df['date'] = df['datetime'].dt.date
            df['time'] = df['datetime'].dt.time
        return df

    @staticmethod
    def Fetch_dividend_yield(tickers_df, exchange):
        combined_data = []
        full_tickers = [f"{ticker}.{exchange}" for ticker in tickers_df]
        for stock in full_tickers:
            try:
                url = f'https://eodhd.com/api/fundamentals/{stock}?api_token={API_Eodhd}&filter=Highlights::DividendYield&fmt=json'
                dividend_yield = requests.get(url).json()
                combined_data.append({"Ticker": stock, "DividendYield(%)": dividend_yield * 100})
            except Exception as e:
                print(f"Error fetching data for {stock}: {e}")
        combined_df = pd.DataFrame(combined_data)
        combined_df = combined_df.dropna(subset=["DividendYield(%)"])
        return combined_df

    @staticmethod
    def Fetch_dividend_share(tickers_df, exchange):
        combined_data = []
        full_tickers = [f"{ticker}.{exchange}" for ticker in tickers_df]
        for stock in full_tickers:
            try:
                url = f'https://eodhd.com/api/fundamentals/{stock}?api_token={API_Eodhd}&filter=Highlights::DividendShare&fmt=json'
                dividend_share = requests.get(url).json()
                combined_data.append({"Ticker": stock, "Dividendshare": dividend_share})
            except Exception as e:
                print(f"Error fetching data for {stock}: {e}")
        combined_df = pd.DataFrame(combined_data)
        combined_df = combined_df.dropna(subset=["Dividendshare"])
        return combined_df

    @staticmethod
    def Fetch_price_momentum(stock, days, api_token=API_Eodhd):
        try:
            end_date = dt.datetime.now()
            start_date = end_date - dt.timedelta(days=365)
            historical_data = EODHD_Functions.Fetch_historical_price(
                stock,
                start_date.strftime("%Y-%m-%d"),
                end_date.strftime("%Y-%m-%d"),
                "d",
                api_token,
                columns=["adjusted_close"]
            )
            historical_data = historical_data.sort_values(by="date")
            if len(historical_data) < days + 1:
                return None
            current_price = historical_data["adjusted_close"].iloc[-1]
            past_price = historical_data["adjusted_close"].iloc[-(days + 1)]
            log_return = np.log(current_price / past_price)
            return log_return
        except Exception as e:
            return None

    @staticmethod
    def Fetch_bond_data(ISIN,api_token=API_Eodhd):
        url = f'https://eodhd.com/api/bond-fundamentals/{ISIN}?api_token={API_Eodhd}&fmt=json'
        response = requests.get(url).json()
        bond_df = pd.DataFrame(response)
        return bond_df
    
    @staticmethod
    def Plot_daily_returns(stock_df):
        stock_df = stock_df.copy()
        stock_df['daily_return'] = stock_df['adjusted_close'].pct_change()
        stock_df['date'] = pd.to_datetime(stock_df['date'])

        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=stock_df['date'],
            y=stock_df['daily_return'] * 100,
            mode='lines',
            name='Daily Returns (%)'
        ))

        fig.update_layout(
            title='Daily Returns',
            xaxis_title='Date',
            yaxis_title='Daily Return (%)',
            xaxis=dict(
                tickformat='%b %Y',
                tickangle=45
            ),
            yaxis=dict(
                tickformat='.2f'
            ),
            template='plotly_white',
            legend=dict(x=0, y=1)
        )

        fig.show()